# Lab: Pretraining Language Models


Now that we have our dataset and tokenizer, in this lab, we will train a language model on a large corpus of text from scratch.


In [2]:
from huggingface_hub import notebook_login

notebook_login()


time: 70.7 ms (started: 2022-11-21 00:59:58 +00:00)


In [9]:
import os
from huggingface_hub import HfApi
from huggingface_hub import HfFolder

token = HfFolder.get_token()
if token is None:
    token = os.environ["HF_USER_ACCESS_TOKEN"]

if token is None:
    raise ValueError("Please login to huggingface_hub")

user_id = HfApi().whoami(token)["name"]

print(f"user id '{user_id}' will be used during this lab")


user id 'entelecheia' will be used during this lab
time: 893 ms (started: 2022-11-21 01:06:46 +00:00)


## BERT Pretraining

In this lab, we will train a BERT-like model using masked-language modeling, one of the two pretraining tasks used in the original BERT paper.

### What is BERT?

BERT is a large-scale language model that was trained on the English Wikipedia using a masked-language modeling objective. The model was then fine-tuned on a variety of downstream tasks, including question answering, natural language inference, and sentiment analysis. BERT was the first large-scale language model to be pre-trained using a deep bidirectional architecture and outperformed previous language models on a variety of tasks.

BERT was originally pre-trained on 1 Million Steps with a global batch size of 256.

> "We train with batch size of 256 sequences (256 sequences \* 512 tokens = 128,000 tokens/batch) for 1,000,000 steps, which is approximately 40 epochs over the 3.3 billion word corpus."

For more information, see the lecture notes on BERT.

### Masked-Language Modeling (MLM)

Masked-language modeling is a pretraining task where we mask some of the input tokens and train the model to predict the original value of the masked tokens. For example, if we have the sentence "The dog ate the apple", we can mask the word "ate" and train the model to predict the original value of the masked token. The model will then learn to predict the original value of the masked tokens based on the context of the sentence.

Example:

> Input: "The dog [MASK] the apple"


## Preprocessing the Dataset

Before training our language model, we need to preprocess our dataset. We will use our tokenizer to tokenize our dataset and then convert the tokens to their IDs. If we have a sentence that is longer than the maximum sequence length, we will truncate the sentence. If the sentence is shorter than the maximum sequence length, we will pad the sentence with the padding token.

Unlike the original BERT paper, we will not use the WordPiece tokenization algorithm. Instead, we will use the `unigram` tokenization algorithm.


In [18]:
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
from tokenizers.processors import BertProcessing

tokenizer_path = "tokenizers/enko_wiki/enko_wiki_unigram_huggingface_vocab_30000.json"
context_length = 512

unigram_tokenizer = Tokenizer.from_file(tokenizer_path)
print(f"Vocab size: {unigram_tokenizer.get_vocab_size()}")
unigram_tokenizer.post_processor = BertProcessing(
    ("</s>", unigram_tokenizer.token_to_id("</s>")),
    ("<s>", unigram_tokenizer.token_to_id("<s>")),
)

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=unigram_tokenizer,
    truncation=True,
    max_length=context_length,
    return_length=True,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="right",
)

print(f"is_fast: {tokenizer.is_fast}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(tokenizer("Hello, my dog is cute"))
tokenizer.save_pretrained(project_dir + "/tokenizers/enko_wiki")


Vocab size: 30000
is_fast: True
Vocab size: 30000
{'input_ids': [1, 8, 14690, 10, 8, 968, 8, 6871, 8, 42, 8, 2777, 72, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


('/content/drive/MyDrive/workspace/projects/ekorpkit-book/tokenizers/enko_wiki/tokenizer_config.json',
 '/content/drive/MyDrive/workspace/projects/ekorpkit-book/tokenizers/enko_wiki/special_tokens_map.json',
 '/content/drive/MyDrive/workspace/projects/ekorpkit-book/tokenizers/enko_wiki/tokenizer.json')

time: 67 ms (started: 2022-11-19 10:55:20 +00:00)


### Load the tokenizer


In [6]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("/tokenizers/enko_wiki")
print(f"is_fast: {tokenizer.is_fast}")


is_fast: True
time: 59.6 ms (started: 2022-11-21 01:03:21 +00:00)


In [11]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")


{'input_ids': [1, 8, 14690, 10, 8, 235, 8, 202, 8, 15219, 489, 2, 8, 37, 8, 235, 8, 15219, 8, 11241, 8, 80, 8, 65, 9, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

time: 20.2 ms (started: 2022-11-21 01:07:07 +00:00)


### Load the dataset


In [5]:
from datasets import load_dataset

data_dir =  "data/tokenizers/enko_filtered_chunk"

dataset = load_dataset("text", data_dir=data_dir, split="train")
dataset


Resolving data files:   0%|          | 0/61 [00:00<?, ?it/s]

Dataset({
    features: ['text'],
    num_rows: 3618972
})

time: 1.02 s (started: 2022-11-19 09:39:39 +00:00)


### Tokenize the dataset


In [12]:
text_column = "text"


def tokenize(element):
    outputs = tokenizer(
        element[text_column],
        truncation=True,
        max_length=context_length,
        return_special_tokens_mask=True,
    )
    return outputs


time: 15.3 ms (started: 2022-11-19 09:55:12 +00:00)


In [ ]:
num_proc = 20

# preprocess dataset
tokenized_dataset = dataset.map(
    tokenize, batched=True, remove_columns=[text_column], num_proc=num_proc
)
tokenized_dataset.features


In [ ]:
from itertools import chain


# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= context_length:
        total_length = (total_length // context_length) * context_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + context_length] for i in range(0, total_length, context_length)]
        for k, t in concatenated_examples.items()
    }
    return result


tokenized_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=num_proc)

# shuffle dataset
tokenized_dataset = tokenized_dataset.shuffle(seed=1234)

print(f"the dataset contains in total {len(tokenized_dataset)*context_length} tokens")
# the dataset contains in total 137,816,832 tokens


We have 137,816,832 tokens in our dataset. For reference, the original BERT paper used 3.2 billion tokens, and GPT-3 uses 300 billion tokens.


## Initializing a New Model

We will initialize a new model using the `bert-base-uncased` configuration. We will then save the configuration to a file so that we can use it later when we load the model.


In [49]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForMaskedLM

tk_path = "tokenizers/enko_wiki"

# Load codeparrot tokenizer trained for Python code tokenization
tokenizer = AutoTokenizer.from_pretrained(tk_path)

# Configuration
config_kwargs = {
    "vocab_size": len(tokenizer),
    "pad_token_id": tokenizer.pad_token_id,
    "mask_token_id": tokenizer.mask_token_id,
    "cls_token_id": tokenizer.cls_token_id,
    "sep_token_id": tokenizer.sep_token_id,
    "unk_token_id": tokenizer.unk_token_id,
}

# # Load model with config and push to hub
config = AutoConfig.from_pretrained("bert-base-uncased", **config_kwargs)
model = AutoModelForMaskedLM.from_config(config)

model_path = "models/enko_wiki_bert_base_uncased"
model.save_pretrained(model_path)


time: 3.4 s (started: 2022-11-19 11:29:40 +00:00)


Our model have 109.1 million parameters just like the original BERT model.


In [45]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"BERT size: {model_size/1000**2:.1f}M parameters")


BERT size: 109.1M parameters
time: 2.07 s (started: 2022-11-19 11:23:15 +00:00)


### Set up a DataCollator

Before we can start training, we need to set up a data collator that will be used to collate the batches of data. We will use the `DataCollatorForLanguageModeling` data collator, which will take care of masking the tokens and padding the sequences.


In [13]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(
    tokenizer, mlm=True, mlm_probability=0.15
)


2022-11-21 01:24:31.069901: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


time: 2.59 s (started: 2022-11-21 01:24:30 +00:00)


### Load the Tokenized Dataset

Our dataset is already tokenized, there are 268,366 examples in our dataset.


In [14]:
from datasets import Dataset

dataset_dir = "data/tokenized_datasets/enko_filtered"

tokenized_dataset = Dataset.load_from_disk(dataset_dir)
tokenized_dataset


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 268366
})

time: 269 ms (started: 2022-11-21 01:24:33 +00:00)


Check the output of the first batch of data.


In [15]:
out = data_collator([tokenized_dataset[i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")


input_ids shape: torch.Size([5, 512])
token_type_ids shape: torch.Size([5, 512])
attention_mask shape: torch.Size([5, 512])
labels shape: torch.Size([5, 512])
time: 70 ms (started: 2022-11-21 01:24:35 +00:00)


## Training the Model

We will configure the training arguments and then set up a trainer to train our model.

### Configure the Training Arguments


In [50]:
from transformers import Trainer, TrainingArguments


args = TrainingArguments(
    output_dir=model_path,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)


Using cuda_amp half precision backend


time: 2.43 s (started: 2022-11-19 11:34:48 +00:00)


### Train the Model


In [51]:
from accelerate import Accelerator

accelerator = Accelerator()
acc_state = {str(k): str(v) for k, v in accelerator.state.__dict__.items()}
device = accelerator.device

print(f"device: {device}")

trainer = accelerator.prepare(trainer)

trainer.train()

trainer.save_model(model_path)


device: cuda
time: 47.9 ms (started: 2022-11-19 11:34:51 +00:00)


It took 6h 33m 0.0s to train our model for 40 epochs.


## Testing the Model

We will load our model and test it on a few examples.


In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

model_path = "models/enko_wiki_bert_base_uncased"

model = AutoModelForMaskedLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)


[INFO|configuration_utils.py:652] 2022-11-22 19:14:02,688 >> loading configuration file /content/drive/MyDrive/workspace/projects/ekorpkit-book/models/enko_wiki_bert_base_uncased/config.json
[INFO|configuration_utils.py:706] 2022-11-22 19:14:02,690 >> Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/workspace/projects/ekorpkit-book/models/enko_wiki_bert_base_uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 4,
  "position_embedding_type": "absolute",
  "sep_token_id": 6,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cach

time: 2.3 s (started: 2022-11-22 19:14:02 +00:00)


In [6]:
# perform predictions
example = "처음으로 대중적으로 <mask> 롤플레잉 게임이다."
for prediction in fill_mask(example):
    print(prediction)


{'score': 0.1881457269191742, 'token': 1183, 'token_str': '만든', 'sequence': '▁ 처음으로 ▁ 대중적 으로 ▁ 만든 ▁ 롤플레잉 ▁ 게임 이다.'}
{'score': 0.14176464080810547, 'token': 3567, 'token_str': '제작한', 'sequence': '▁ 처음으로 ▁ 대중적 으로 ▁ 제작한 ▁ 롤플레잉 ▁ 게임 이다.'}
{'score': 0.04182405769824982, 'token': 15022, 'token_str': '롤플레잉', 'sequence': '▁ 처음으로 ▁ 대중적 으로 ▁ 롤플레잉 ▁ 롤플레잉 ▁ 게임 이다.'}
{'score': 0.035137832164764404, 'token': 3225, 'token_str': '개발한', 'sequence': '▁ 처음으로 ▁ 대중적 으로 ▁ 개발한 ▁ 롤플레잉 ▁ 게임 이다.'}
{'score': 0.033710286021232605, 'token': 3171, 'token_str': '아시안', 'sequence': '▁ 처음으로 ▁ 대중적 으로 ▁ 아시안 ▁ 롤플레잉 ▁ 게임 이다.'}
time: 223 ms (started: 2022-11-22 19:14:05 +00:00)
